In [12]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd

In [13]:
# Store episodes
episodes = []

# Looping all season
for sn in range(1,7):
    # Get content from address for each season
    response = requests.get('https://www.imdb.com/title/tt2575988/episodes?season='+str(sn))
    # Parse the html
    html_soup = BeautifulSoup(response.text, 'html5lib')
    # Find the <div class = "info"> section
    episode_containers = html_soup.find_all('div', class_='info')
    # Loop throught all episodes
    for episode in episode_containers:
        season = sn
        # Find each tag with specific class or name
        episodeNum = episode.meta['content']
        episodeTitle = episode.a['title']

        # Get episode link from title href
        link = episode.a['href']
        # Add domain
        completeLink = "https://www.imdb.com" + link
        # Get content from link
        getChar = requests.get(completeLink)
        # Parse
        charParsing = BeautifulSoup(getChar.text, 'html5lib')
        # Look for <div class = "sc-36c36dd0-6 ewJBXI"> --> this is the cast section
        characters = charParsing.find_all('div', class_ = "sc-36c36dd0-6 ewJBXI")
        # To store cast names
        names = []

        # Loop for each character
        for cast in characters:
            # Find <div class = "sc-36c36dd0-8 fSYMLK"> --> get the specific cast section
            chara = cast.find_all('div', class_ = "sc-36c36dd0-8 fSYMLK")
            # Loop for each section, look for text
            for name in chara:
                # Find <div class = "sc-36c36dd0-1 QSQgP"> --> get the name text from a cast
                actorName = name.find('a', class_ = "sc-36c36dd0-1 QSQgP").text
                # Append name to names array
                names.append(actorName)

        # Get text or value from a tag
        airdate = episode.find('div', class_ = "airdate").text.strip()
        rating = episode.find('span', class_ = "ipl-rating-star__rating").text
        totalVotes = episode.find('span', class_ = "ipl-rating-star__total-votes").text
        desc = episode.find('div', class_ = 'item_description').text.strip()
        # Store variables for each episode
        episode_data = [season, episodeNum, episodeTitle, airdate, rating, totalVotes, desc, names]
        # Append episode info to giant array
        episodes.append(episode_data)
# Headers with correponding data
episodes = pd.DataFrame(episodes, columns= ['season', 'episode_number', 'title', 'airdate', 'rating', 'total_votes', 'desc', 'cast'])
# Use it as the header
episodes.head()

,season,episode_number,title,airdate,rating,total_votes,desc,cast
0,1,1,Minimum Viable Product,6 Apr. 2014,7.7,"(2,766)",Richard is a computer programmer. He has to ch...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
1,1,2,The Cap Table,13 Apr. 2014,7.7,"(2,229)",Richard hires Jared to design a business plan....,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
2,1,3,Articles of Incorporation,20 Apr. 2014,8.0,"(2,145)",The guys find themselves in trouble with their...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
3,1,4,Fiduciary Duties,27 Apr. 2014,7.7,"(1,996)",Richard drunkenly promises to make Erlich a bo...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
4,1,5,Signaling Risk,4 May 2014,8.3,"(2,045)",Now that Pied Piper is the official company na...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."


In [14]:
# Function to clean total votes
def cleaning(votes):
    for r in ((',',''), ('(',''),(')','')):
        votes = votes.replace(*r)
    return votes

# For 'total_votes' section, use the apply method to use the cleaning function
episodes['total_votes'] = episodes.total_votes.apply(cleaning).astype(int)
episodes.head()

,season,episode_number,title,airdate,rating,total_votes,desc,cast
0,1,1,Minimum Viable Product,6 Apr. 2014,7.7,2766,Richard is a computer programmer. He has to ch...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
1,1,2,The Cap Table,13 Apr. 2014,7.7,2229,Richard hires Jared to design a business plan....,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
2,1,3,Articles of Incorporation,20 Apr. 2014,8.0,2145,The guys find themselves in trouble with their...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
3,1,4,Fiduciary Duties,27 Apr. 2014,7.7,1996,Richard drunkenly promises to make Erlich a bo...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."
4,1,5,Signaling Risk,4 May 2014,8.3,2045,Now that Pied Piper is the official company na...,"[Thomas Middleditch, T.J. Miller, Josh Brener,..."


In [15]:

# Convert the data stored in episodes to csv file
episodes.to_csv('SilliconValley_IMDb_cast.csv', index = False)